In [4]:
import jieba.analyse
import xlwings as xw

wb = xw.Book('weibo_clean.xls')
shts = wb.sheets

# we divide the posts into 5 periods: 12/20 - 1/22, 1/23 - 2/18, 2/19 - 3/11, 3/12 - 4/7, 4/8 - 4/30
p1, p2, p3, p4, p5 = [], [], [], [], []
all_P = []
for sht in shts:
    date = sht.name
    posts = sht.range('A1').expand('down').value
    all_P.append(posts)
for p in all_P[:34]:
    p1 += p
for p in all_P[34:61]:
    p2 += p
for p in all_P[61:83]:
    p3 += p
for p in all_P[83:110]:
    p4 += p
for p in all_P[110:132]:
    p5 += p


In [32]:
from gensim import corpora, models

stop_words = [' ', '…', '转发', ':', '.', '冠状病毒', '新型', 'Repost', '（', 'O']
with open('cn_stopwords.txt', 'r') as f:
    for line in f:
        stop_words.append(line.strip())
        
def topic_model(text):
    words_ls = []
    for p in p1:
        if p:
            words = [w for w in jieba.cut(p) if w not in stop_words]
            words_ls.append(words)

    dictionary = corpora.Dictionary(words_ls)
    corpus = [dictionary.doc2bow(words) for words in words_ls]
    lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=5)
    for topic in lda.print_topics(num_words=6):
        print(topic)
        
for i, each in enumerate([[p1 + p2 + p3], [p4 + p5]]):
    print('The 5 topics in period %s:' % str(i+1))
    topic_model(each)


The 5 topics in period 1:
(0, '0.385*"保护" + 0.324*"注意" + 0.083*"可怕" + 0.014*"谢谢" + 0.009*"扩散" + 0.008*"潜伏期"')
(1, '0.145*"保护" + 0.051*"注意" + 0.042*"扩散" + 0.033*"可怕" + 0.029*"正常" + 0.014*"病毒"')
(2, '0.931*"注意" + 0.010*"N95" + 0.002*"口罩" + 0.002*"保护" + 0.001*"肺炎" + 0.001*"武汉"')
(3, '0.113*"注意" + 0.035*"年" + 0.030*"保护" + 0.025*"肺炎" + 0.019*"hands" + 0.019*"folded"')
(4, '0.981*"注意安全" + 0.005*"保护" + 0.003*"扩散" + 0.001*"肺炎" + 0.000*"年" + 0.000*"例"')
The 5 topics in period 2:
(0, '0.433*"平安" + 0.022*"武汉" + 0.021*"保护" + 0.016*"情况" + 0.011*"注意" + 0.008*"说"')
(1, '0.552*"平安" + 0.203*"辛苦" + 0.155*"保护" + 0.007*"武汉" + 0.005*"hands" + 0.005*"folded"')
(2, '0.240*"信" + 0.051*"保护" + 0.026*"武汉" + 0.018*"注意" + 0.016*"情况" + 0.014*"口罩"')
(3, '0.764*"天呐" + 0.052*"平安" + 0.026*"保护" + 0.019*"情况" + 0.012*"武汉" + 0.005*"肺炎"')
(4, '0.125*"平安" + 0.063*"肺炎" + 0.062*"武汉" + 0.055*"例" + 0.044*"情况" + 0.016*"日"')
